In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error

In [2]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')
test_joke_df_nofactrating = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

In [3]:
train_joke_df["UID"] = train_joke_df["UID"].astype(int)
train_joke_df["JID"] = train_joke_df["JID"].astype(int)

In [4]:
train_joke_df["UID"] = train_joke_df["UID"] - 1
train_joke_df["JID"] = train_joke_df["JID"] - 1

In [5]:
npusers_arr = np.zeros((24983, 100), dtype=np.half)

for row in tqdm(train_joke_df.values):
    npusers_arr[int(row[0]), int(row[1])] = row[2]

  0%|          | 0/1448364 [00:00<?, ?it/s]

In [6]:
users_df = pd.DataFrame(npusers_arr, index=range(0, npusers_arr.shape[0]), columns=[f'j_{i}' for i in range(100)])
users_df

,j_0,j_1,j_2,j_3,j_4,j_5,j_6,j_7,j_8,j_9,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
0,-7.820312,8.789062,-9.656250,-8.156250,-7.519531,-8.500000,-9.851562,4.171875,-8.976562,-4.761719,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.628906,0.000000,0.000000,0.000000
1,4.078125,-0.290039,6.359375,4.371094,0.000000,-9.656250,-0.729980,-5.339844,8.882812,9.218750,...,2.820312,-4.949219,-0.290039,7.859375,-0.189941,0.000000,3.060547,0.000000,-4.320312,0.000000
2,0.000000,0.000000,0.000000,0.000000,9.031250,9.273438,9.031250,9.273438,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,8.351562,0.000000,0.000000,1.799805,0.000000,-2.820312,6.210938,0.000000,1.839844,...,0.000000,0.000000,0.000000,0.529785,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,8.500000,4.609375,-4.171875,-5.390625,0.000000,0.000000,7.039062,0.000000,-0.439941,0.000000,...,0.000000,5.578125,4.269531,5.191406,5.730469,1.549805,0.000000,6.550781,1.799805,1.599609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.439941,0.000000,0.000000,2.330078,0.000000,6.750000,-8.789062,-0.529785,0.000000,0.000000,...,8.828125,-1.209961,9.218750,-6.699219,0.000000,9.031250,6.550781,8.687500,0.000000,7.429688
24979,0.000000,-8.156250,8.593750,9.078125,0.870117,-8.929688,-3.500000,5.781250,-8.109375,0.000000,...,-1.169922,-5.730469,0.000000,0.239990,9.218750,-8.203125,0.000000,-8.593750,9.132812,8.453125
24980,0.000000,0.000000,0.000000,0.000000,-7.769531,0.000000,0.000000,-6.750000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24981,0.000000,0.000000,0.000000,0.000000,-9.710938,0.000000,4.558594,-8.296875,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
train_joke_df = train_joke_df.merge(users_df, how='inner', left_on='UID',  right_index=True)
train_joke_df

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
0,18028,5,-1.26,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
18990,18028,39,8.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
50486,18028,6,3.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
69643,18028,15,-9.27,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
124589,18028,14,8.11,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303561,21378,61,0.15,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1331118,21378,45,1.31,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1368576,21378,48,-0.68,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1376073,21378,53,2.33,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0


In [8]:
for i, row in enumerate(tqdm(train_joke_df.values)):
    j = int(row[1])
    #print(train_joke_df.iloc[i, 3 + j])
    assert np.isclose(row[2], train_joke_df.iloc[i, 3 + j], atol=1e-2)
    train_joke_df.iloc[i, 3 + j] = 0
    #print(train_joke_df.iloc[i, 3 + j])


  0%|          | 0/1448364 [00:00<?, ?it/s]

In [9]:
for i, row in enumerate(tqdm(train_joke_df.values)):
    j = int(row[1])
    #print(train_joke_df.iloc[i, 3 + j])
    #print(row[3 + joke_id])
    assert np.isclose(0, train_joke_df.iloc[i, 3 + j])


  0%|          | 0/1448364 [00:00<?, ?it/s]

In [10]:
train_joke_df

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
0,18028,5,-1.26,0.0,0.0,0.0,0.0,1.120117,0.000000,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
18990,18028,39,8.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
50486,18028,6,3.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
69643,18028,15,-9.27,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
124589,18028,14,8.11,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303561,21378,61,0.15,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1331118,21378,45,1.31,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1368576,21378,48,-0.68,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1376073,21378,53,2.33,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0


In [11]:
train_joke_df.tail(30)

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
460572,21378,28,3.40,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
485937,21378,33,-4.61,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
488228,21378,6,-8.01,0.0,0.0,0.0,0.0,-2.820312,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
509988,21378,60,-2.62,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
579667,21378,25,1.50,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
662520,21378,67,-0.53,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
702969,21378,34,2.28,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
703279,21378,41,1.31,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
739560,21378,15,-7.43,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
831839,21378,38,-3.35,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0


In [12]:
rows, r_pos = np.unique(train_joke_df.values[:,0], return_inverse=True)
cols, c_pos = np.unique(train_joke_df.values[:,1], return_inverse=True)

In [13]:
rows

array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 2.4980e+04, 2.4981e+04,
       2.4982e+04])

In [14]:
cols

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94., 95., 96., 97., 98., 99.])

In [15]:
train_df, valid_df, train_queries, valid_queries = train_test_split(train_joke_df, r_pos, test_size=0.3, random_state=42)

In [16]:
valid_df[valid_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
51504,44,71,3.20,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
693981,44,22,8.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1239916,44,76,3.35,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
826257,44,6,4.51,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.000000,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1089514,44,78,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
226234,44,79,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1413678,44,80,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
643300,44,92,3.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,0.000000,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
992047,44,66,8.45,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
585603,44,57,0.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961


In [17]:
train_df[train_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
504867,44,2,5.49,8.789062,0.0,0.000000,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1214727,44,44,7.52,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1421955,44,73,1.02,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
919959,44,21,5.58,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
553877,44,20,6.41,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650346,44,91,3.88,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,0.000000,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1230384,44,30,5.39,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
746078,44,50,5.15,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
258762,44,86,2.48,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961


In [25]:
for i, row in enumerate(tqdm(valid_df.values)):
    u = int(row[0])
    j = int(row[1])
    
    train_df.loc[train_df['UID'] == u, f'j_{j}'] = 0
    valid_df.loc[valid_df['UID'] == u, f'j_{j}'] = 0

  0%|          | 0/434510 [00:00<?, ?it/s]

In [19]:
train_df[train_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
504867,44,2,5.49,8.789062,0.0,0.000000,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
1214727,44,44,7.52,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
1421955,44,73,1.02,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
919959,44,21,5.58,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
553877,44,20,6.41,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650346,44,91,3.88,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
1230384,44,30,5.39,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
746078,44,50,5.15,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
258762,44,86,2.48,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961


In [26]:
valid_df[valid_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
51504,44,71,3.20,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
693981,44,22,8.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
1239916,44,76,3.35,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
826257,44,6,4.51,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
1089514,44,78,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
226234,44,79,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
1413678,44,80,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
643300,44,92,3.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
992047,44,66,8.45,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961
585603,44,57,0.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.0,...,0.0,3.880859,0.0,0.0,0.290039,0.0,1.459961,-2.769531,2.179688,-1.209961


In [27]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [28]:
train_df.drop(columns=['Rating'])

,UID,JID,j_0,j_1,j_2,j_3,j_4,j_5,j_6,j_7,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
1393815,3129,51,0.000000,0.000000,0.000000,0.000000,0.000000,7.859375,0.000000,5.441406,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
717533,411,21,0.000000,0.000000,0.000000,0.000000,-1.700195,-2.039062,0.290039,-4.808594,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
740045,21467,60,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.289062,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
258598,18326,16,1.019531,0.000000,0.000000,0.000000,0.000000,-1.070312,-4.808594,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.800781,-0.189941,-6.601562
28158,22924,59,-0.970215,0.340088,-3.640625,-5.390625,0.000000,0.000000,0.000000,0.000000,...,-3.539062,0.000000,0.000000,-3.300781,-1.169922,0.680176,0.239990,0.000000,-0.779785,-2.519531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193841,24455,41,0.000000,2.230469,0.000000,1.650391,0.919922,3.500000,0.000000,1.209961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
768014,864,20,0.000000,0.000000,0.000000,0.000000,3.199219,0.000000,4.761719,4.171875,...,0.000000,0.000000,0.000000,0.000000,1.839844,0.000000,0.000000,0.000000,0.000000,0.000000
661501,21881,62,0.000000,0.000000,0.000000,0.000000,2.039062,8.828125,-0.870117,0.000000,...,0.000000,5.730469,8.828125,0.000000,8.828125,0.000000,7.281250,8.250000,0.000000,-0.290039
1192804,24548,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.140625,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
cat_features = ['UID', 'JID']

In [30]:
train_pool = Pool(train_df.drop(columns=['Rating']), label=train_df['Rating'], cat_features=cat_features)
valid_pool = Pool(valid_df.drop(columns=['Rating']), label=valid_df['Rating'], cat_features=cat_features)
main_pool = Pool(train_joke_df.drop(columns=['Rating']), label=train_joke_df['Rating'], cat_features=cat_features)

test_pool = Pool(test_joke_df_nofactrating, cat_features=cat_features)

In [31]:
#train = Pool(
#    data=X_train,
#    label=y_train,
#    group_id=queries_train,
#    cat_features=[0, 1]
#)

#test = Pool(
#    data=X_test,
#    label=y_test,
#    group_id=queries_test,
#    cat_features=[0, 1]
#)

In [32]:
default_parameters = {
    'iterations': 3000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}


In [33]:
model = CatBoostRegressor(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.077231
0:	learn: 5.1492138	test: 5.1458319	best: 5.1458319 (0)	total: 499ms	remaining: 24m 55s
1:	learn: 5.0754650	test: 5.0655625	best: 5.0655625 (1)	total: 778ms	remaining: 19m 25s
2:	learn: 5.0113680	test: 4.9940474	best: 4.9940474 (2)	total: 940ms	remaining: 15m 38s
3:	learn: 4.9548294	test: 4.9306176	best: 4.9306176 (3)	total: 1.1s	remaining: 13m 42s
4:	learn: 4.9042618	test: 4.8739502	best: 4.8739502 (4)	total: 1.26s	remaining: 12m 33s
5:	learn: 4.8614843	test: 4.8258283	best: 4.8258283 (5)	total: 1.42s	remaining: 11m 47s
6:	learn: 4.8228825	test: 4.7822445	best: 4.7822445 (6)	total: 1.7s	remaining: 12m 7s
7:	learn: 4.7891507	test: 4.7450098	best: 4.7450098 (7)	total: 1.87s	remaining: 11m 38s
8:	learn: 4.7586267	test: 4.7115978	best: 4.7115978 (8)	total: 2.08s	remaining: 11m 30s
9:	learn: 4.7310479	test: 4.6812792	best: 4.6812792 (9)	total: 2.25s	remaining: 11m 13s
10:	learn: 4.7075881	test: 4.6542599	best: 4.6542599 (10)	total: 2.58s	remaining: 11m 41s
11:

92:	learn: 4.2367580	test: 4.3649978	best: 4.3649978 (92)	total: 35.7s	remaining: 18m 35s
93:	learn: 4.2336352	test: 4.3644108	best: 4.3644108 (93)	total: 36.1s	remaining: 18m 37s
94:	learn: 4.2310430	test: 4.3641896	best: 4.3641896 (94)	total: 36.5s	remaining: 18m 34s
95:	learn: 4.2280297	test: 4.3638337	best: 4.3638337 (95)	total: 36.8s	remaining: 18m 34s
96:	learn: 4.2248792	test: 4.3631728	best: 4.3631728 (96)	total: 37.3s	remaining: 18m 36s
97:	learn: 4.2227964	test: 4.3626552	best: 4.3626552 (97)	total: 37.9s	remaining: 18m 42s
98:	learn: 4.2203898	test: 4.3620239	best: 4.3620239 (98)	total: 38.4s	remaining: 18m 44s
99:	learn: 4.2165126	test: 4.3618489	best: 4.3618489 (99)	total: 39s	remaining: 18m 51s
100:	learn: 4.2141588	test: 4.3615908	best: 4.3615908 (100)	total: 39.5s	remaining: 18m 52s
101:	learn: 4.2113586	test: 4.3615536	best: 4.3615536 (101)	total: 39.8s	remaining: 18m 50s
102:	learn: 4.2093267	test: 4.3613617	best: 4.3613617 (102)	total: 40.1s	remaining: 18m 48s
103:	l

182:	learn: 4.0115561	test: 4.3385084	best: 4.3384308 (181)	total: 1m 15s	remaining: 19m 16s
183:	learn: 4.0087037	test: 4.3386512	best: 4.3384308 (181)	total: 1m 15s	remaining: 19m 16s
184:	learn: 4.0060147	test: 4.3385798	best: 4.3384308 (181)	total: 1m 15s	remaining: 19m 15s
185:	learn: 4.0043195	test: 4.3382337	best: 4.3382337 (185)	total: 1m 16s	remaining: 19m 15s
186:	learn: 4.0026164	test: 4.3379099	best: 4.3379099 (186)	total: 1m 16s	remaining: 19m 14s
187:	learn: 3.9999026	test: 4.3377745	best: 4.3377745 (187)	total: 1m 17s	remaining: 19m 13s
188:	learn: 3.9952678	test: 4.3379857	best: 4.3377745 (187)	total: 1m 17s	remaining: 19m 14s
189:	learn: 3.9919237	test: 4.3376434	best: 4.3376434 (189)	total: 1m 17s	remaining: 19m 12s
190:	learn: 3.9898025	test: 4.3375169	best: 4.3375169 (190)	total: 1m 18s	remaining: 19m 11s
191:	learn: 3.9875217	test: 4.3371388	best: 4.3371388 (191)	total: 1m 18s	remaining: 19m 9s
192:	learn: 3.9859101	test: 4.3368053	best: 4.3368053 (192)	total: 1m 1

271:	learn: 3.8227307	test: 4.3186630	best: 4.3186262 (270)	total: 1m 50s	remaining: 18m 25s
272:	learn: 3.8216661	test: 4.3185235	best: 4.3185235 (272)	total: 1m 50s	remaining: 18m 24s
273:	learn: 3.8185642	test: 4.3184631	best: 4.3184631 (273)	total: 1m 50s	remaining: 18m 23s
274:	learn: 3.8172829	test: 4.3183453	best: 4.3183453 (274)	total: 1m 51s	remaining: 18m 22s
275:	learn: 3.8144662	test: 4.3182349	best: 4.3182349 (275)	total: 1m 51s	remaining: 18m 21s
276:	learn: 3.8123567	test: 4.3180595	best: 4.3180595 (276)	total: 1m 51s	remaining: 18m 20s
277:	learn: 3.8104441	test: 4.3178852	best: 4.3178852 (277)	total: 1m 52s	remaining: 18m 19s
278:	learn: 3.8081076	test: 4.3177236	best: 4.3177236 (278)	total: 1m 52s	remaining: 18m 18s
279:	learn: 3.8062475	test: 4.3176075	best: 4.3176075 (279)	total: 1m 52s	remaining: 18m 17s
280:	learn: 3.8042721	test: 4.3177334	best: 4.3176075 (279)	total: 1m 53s	remaining: 18m 18s
281:	learn: 3.8020827	test: 4.3174784	best: 4.3174784 (281)	total: 1m 

...

1951:	learn: 2.3482184	test: 4.3847426	best: 4.2921027 (736)	total: 13m 26s	remaining: 7m 12s
1952:	learn: 2.3480040	test: 4.3847727	best: 4.2921027 (736)	total: 13m 26s	remaining: 7m 12s
1953:	learn: 2.3471494	test: 4.3848619	best: 4.2921027 (736)	total: 13m 27s	remaining: 7m 12s
1954:	learn: 2.3470070	test: 4.3849607	best: 4.2921027 (736)	total: 13m 27s	remaining: 7m 11s
1955:	learn: 2.3468398	test: 4.3850522	best: 4.2921027 (736)	total: 13m 28s	remaining: 7m 11s
1956:	learn: 2.3465470	test: 4.3851113	best: 4.2921027 (736)	total: 13m 28s	remaining: 7m 10s
1957:	learn: 2.3450675	test: 4.3852266	best: 4.2921027 (736)	total: 13m 28s	remaining: 7m 10s
1958:	learn: 2.3446055	test: 4.3852699	best: 4.2921027 (736)	total: 13m 29s	remaining: 7m 10s
1959:	learn: 2.3442273	test: 4.3853073	best: 4.2921027 (736)	total: 13m 29s	remaining: 7m 9s
1960:	learn: 2.3433944	test: 4.3853080	best: 4.2921027 (736)	total: 13m 30s	remaining: 7m 9s
1961:	learn: 2.3425968	test: 4.3853411	best: 4.2921027 (736)	t

2039:	learn: 2.3027399	test: 4.3911726	best: 4.2921027 (736)	total: 14m 4s	remaining: 6m 37s
2040:	learn: 2.3023830	test: 4.3912374	best: 4.2921027 (736)	total: 14m 4s	remaining: 6m 36s
2041:	learn: 2.3010716	test: 4.3913083	best: 4.2921027 (736)	total: 14m 5s	remaining: 6m 36s
2042:	learn: 2.3003713	test: 4.3913840	best: 4.2921027 (736)	total: 14m 5s	remaining: 6m 36s
2043:	learn: 2.3001331	test: 4.3914135	best: 4.2921027 (736)	total: 14m 6s	remaining: 6m 35s
2044:	learn: 2.2998068	test: 4.3914860	best: 4.2921027 (736)	total: 14m 6s	remaining: 6m 35s
2045:	learn: 2.2994863	test: 4.3916342	best: 4.2921027 (736)	total: 14m 6s	remaining: 6m 34s
2046:	learn: 2.2988576	test: 4.3916854	best: 4.2921027 (736)	total: 14m 7s	remaining: 6m 34s
2047:	learn: 2.2984643	test: 4.3917794	best: 4.2921027 (736)	total: 14m 8s	remaining: 6m 34s
2048:	learn: 2.2981350	test: 4.3919697	best: 4.2921027 (736)	total: 14m 8s	remaining: 6m 33s
2049:	learn: 2.2974718	test: 4.3919903	best: 4.2921027 (736)	total: 14

2127:	learn: 2.2653241	test: 4.3992351	best: 4.2921027 (736)	total: 14m 42s	remaining: 6m 1s
2128:	learn: 2.2651839	test: 4.3992921	best: 4.2921027 (736)	total: 14m 42s	remaining: 6m 1s
2129:	learn: 2.2646287	test: 4.3992354	best: 4.2921027 (736)	total: 14m 43s	remaining: 6m
2130:	learn: 2.2641952	test: 4.3992900	best: 4.2921027 (736)	total: 14m 43s	remaining: 6m
2131:	learn: 2.2638867	test: 4.3994442	best: 4.2921027 (736)	total: 14m 44s	remaining: 5m 59s
2132:	learn: 2.2633689	test: 4.3996315	best: 4.2921027 (736)	total: 14m 44s	remaining: 5m 59s
2133:	learn: 2.2630777	test: 4.3996270	best: 4.2921027 (736)	total: 14m 44s	remaining: 5m 59s
2134:	learn: 2.2629347	test: 4.3997947	best: 4.2921027 (736)	total: 14m 45s	remaining: 5m 58s
2135:	learn: 2.2628259	test: 4.3998875	best: 4.2921027 (736)	total: 14m 45s	remaining: 5m 58s
2136:	learn: 2.2625859	test: 4.3998879	best: 4.2921027 (736)	total: 14m 46s	remaining: 5m 57s
2137:	learn: 2.2619857	test: 4.3999239	best: 4.2921027 (736)	total: 14

2215:	learn: 2.2280333	test: 4.4059087	best: 4.2921027 (736)	total: 15m 20s	remaining: 5m 25s
2216:	learn: 2.2276618	test: 4.4058893	best: 4.2921027 (736)	total: 15m 21s	remaining: 5m 25s
2217:	learn: 2.2274356	test: 4.4059904	best: 4.2921027 (736)	total: 15m 21s	remaining: 5m 24s
2218:	learn: 2.2266477	test: 4.4062293	best: 4.2921027 (736)	total: 15m 22s	remaining: 5m 24s
2219:	learn: 2.2260602	test: 4.4063590	best: 4.2921027 (736)	total: 15m 22s	remaining: 5m 24s
2220:	learn: 2.2255379	test: 4.4063375	best: 4.2921027 (736)	total: 15m 22s	remaining: 5m 23s
2221:	learn: 2.2247566	test: 4.4063659	best: 4.2921027 (736)	total: 15m 23s	remaining: 5m 23s
2222:	learn: 2.2245593	test: 4.4064063	best: 4.2921027 (736)	total: 15m 23s	remaining: 5m 22s
2223:	learn: 2.2241596	test: 4.4064545	best: 4.2921027 (736)	total: 15m 24s	remaining: 5m 22s
2224:	learn: 2.2240086	test: 4.4065191	best: 4.2921027 (736)	total: 15m 24s	remaining: 5m 22s
2225:	learn: 2.2235501	test: 4.4065055	best: 4.2921027 (736)

KeyboardInterrupt: 

In [ ]:
assert False

In [ ]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


In [ ]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost0.9.csv')

In [ ]:
parameters = {
    'iterations': 2000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}

model = CatBoostRanker(**parameters)
model.fit(main_pool, eval_set=valid_pool, plot=True)

In [ ]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


In [ ]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost.csv')